## Create Genesis Branch

This notebooks creates a genesis branch of type "twig"

First we switch to the root folder.
Then we load necessary modules, functions and variables. Then we load the keyword arguments for the function call, as someone would enter them for an rpc call. They are checked for validity. Then the base64 encoded entries and other entries are converted into bytes form using the schema of the function call. Then the function call is made. Its response is then re-translated into base64 (at least all the values that are bytes).  

In [1]:
import os 
os.chdir('..')

In [2]:
import lakat.branch.functions as lakat_branch_functions
import lakat.branch.schema as lakat_branch_schema
from utils.format.schema import check_argument, convert_to_bytes_based_on_schema, convert_from_bytes_based_on_schema
from utils.encode.bytes import encode_bytes_to_base64_str, decode_base64_str_to_bytes
from config.branch_cfg import PROPER_BRANCH_TYPE_ID, TWIG_BRANCH_TYPE_ID

In [3]:
# some fake signature encoded in bytes64 
signature = encode_bytes_to_base64_str(bytes(0))
accept_conflicts = False
msg = 'Genesis Submit'
name = 'Genesis Branch'
kwargs = dict(branch_type=TWIG_BRANCH_TYPE_ID, name=name, signature=signature, accept_conflicts=accept_conflicts, msg=msg)
print('keyword arguments: ', kwargs)

keyword arguments:  {'branch_type': 1, 'name': 'Genesis Branch', 'signature': '', 'accept_conflicts': False, 'msg': 'Genesis Submit'}


### Create genesis branch without RPC server call

In [4]:
# check if the arguments are valid
check_argument(arg=kwargs, schema=lakat_branch_schema.create_genesis_branch_call)

True

In [5]:
# convert string and base64 encoded arguments to bytes
converted_kwargs = convert_to_bytes_based_on_schema(schema=lakat_branch_schema.create_genesis_branch_call, data=kwargs)
print("converted_kwargs", converted_kwargs)

converted_kwargs {'branch_type': 1, 'name': b'\x01\x0eGenesis Branch', 'signature': b'', 'accept_conflicts': False, 'msg': b'\x01\x0eGenesis Submit'}


In [6]:
response = lakat_branch_functions.create_genesis_branch(**converted_kwargs)

In [7]:
# response is the branch id of the genesis branch. We can use the schema to decode the response
decoded_response = convert_from_bytes_based_on_schema(schema=lakat_branch_schema.create_genesis_branch_response, data=response)
print("original response: ", response, "\ndecoded  response: ", decoded_response)

original response:  b'\x01Q\x12\x02O\xa4' 
decoded  response:  AVESAk+k


### Create genesis branch via RPC server call

First start the server via `python main.py` (first need to install the dependencies from the *requirements.txt*) or start the docker container exposed on port `3355`.

In [15]:
import requests
import json
from config.rpc_cfg import RPC_PORT

In [16]:
def json_rpc_call(method, params=None):
    url = f"http://localhost:{RPC_PORT}/"
    headers = {'content-type': 'application/json'}
    payload = {
        "method": method,
        "params": params,
        "jsonrpc": "2.0",
        "id": 0,
    }
    encoded_payload = json.dumps(payload).encode('utf-8')
    response = requests.post(url, data=encoded_payload, headers=headers)
    # print(response)#
    return response.json()

In [17]:
# convert the keyword arguments to a list of values for the rpc call
rpc_call_list = list(kwargs.values())
# make the rpc call and get the response
response = json_rpc_call(method="create_genesis_branch", params=rpc_call_list)
# response is inside the result field
branch_head_id = response['result']
print("Test 'create_genesis_branch' passed with branch ID:", branch_head_id)

Test 'create_genesis_branch' passed with branch ID: AVESAiwg
